# Изучение закономерностей, определяющих успешность игр

**Описание исследования**

Интернет-магазин «Стримчик» продаёт по всему миру компьютерные игры. Из открытых источников доступны исторические данные о продажах игр, оценки пользователей и экспертов, жанры и платформы (например, Xbox или PlayStation)

_____
**Цели исследования**

* выявить определяющие успешность игры закономерности.
* получить возможность делать ставку на потенциально популярный продукт и планировать рекламные кампании.
_____
**Задачи исследования.**

*  провести исследовательский анализ данных
*  составить портрет пользователя каждого региона
*  выявить определяющие успешность игры закономерности.

**Исходные данные.**
    
Перед нами данные до 2016 года, где:

*	Name — название игры
*	Platform — платформа
*	Year_of_Release — год выпуска
*	Genre — жанр игры
*	NA_sales — продажи в Северной Америке (миллионы проданных копий)
*	EU_sales — продажи в Европе (миллионы проданных копий)
*	JP_sales — продажи в Японии (миллионы проданных копий)
*	Other_sales — продажи в других странах (миллионы проданных копий)
*	Critic_Score — оценка критиков (максимум 100)
*	User_Score — оценка пользователей (максимум 10)
*	Rating — рейтинг от организации ESRB (англ. Entertainment Software Rating Board). Эта ассоциация определяет рейтинг компьютерных игр и присваивает им подходящую возрастную категорию



______

**Данное исследование разделим на несколько частей:**

* [Изучение общей информации](#main_info)
* [Подготовка данных](#preparing)
* [Исследовательский анализ данных](#explore_data)
* [Портрет пользователя каждого региона](#portret)
* [Проверка гипотез](#check_hypothysis)
* [Общий вывод](#summary)

<a id='main_info'></a>
# 1. Изучение общей информации

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from scipy import stats as st

pd.set_option('display.float_format', '{:,.2f}'.format)

In [3]:
data = pd.read_csv('games.csv')
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16715 entries, 0 to 16714
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Name             16713 non-null  object 
 1   Platform         16715 non-null  object 
 2   Year_of_Release  16446 non-null  float64
 3   Genre            16713 non-null  object 
 4   NA_sales         16715 non-null  float64
 5   EU_sales         16715 non-null  float64
 6   JP_sales         16715 non-null  float64
 7   Other_sales      16715 non-null  float64
 8   Critic_Score     8137 non-null   float64
 9   User_Score       10014 non-null  object 
 10  Rating           9949 non-null   object 
dtypes: float64(6), object(5)
memory usage: 1.4+ MB


Выведем уникальные значения по всем столбцам - посмотрим есть ли какие-то аномалии

In [ ]:
# выведем уникальные значения по всем столбцам - посмотрим есть ли какие-то аномалии
def show_data_values(data):
    for col in data.columns:    
        try:
            values = sorted(data[col].unique())
            # если больше 10 значений, выведем только первые и последние 5
            if len(values) > 10:
                display (col, values[:5], values[-5:])
            else: display(col, values)
        except:
            display(f'После except: {col}', data[col].unique()) 
            
            
show_data_values(data)

***Вывод:*** Найдены следующие аномалии:
* названия столбцов в написаны разном регистре
* в 'User_Score' есть нечисловое значение 'tbd' 
* в 'User_Score' тип столюбца нужно заменить на числовой

<a id='preparing'></a>
# 2. Подготовка данных

**На этом шаге**
* исправим аномалии
* обработаем пропуски
* добавим столбец с суммарными продажами во всех регионах

Приведем названия столбцов в нижний регистр

In [ ]:
data.columns = data.columns.str.lower()
data.columns

Ячейки, где значения 'user_score' равно 'tbd' заполним как 'nan', потому что это аббревиатура, означающая, что еще не определено значение.

In [ ]:
data.loc[data['user_score'] == 'tbd', 'user_score' ] = float('nan')
data['user_score'] = pd.to_numeric(data['user_score'])


**Обработка пропусков**

Теперь проверим пропуски

In [ ]:
data.isna().sum()

Заполним пропущенные значения рейтингов на 'unknown', так как восстановить их не получится, а эта информация принцыпиальна для исследования

In [ ]:
data['critic_score'] = data['critic_score'].fillna(-1)
data['user_score'] = data['user_score'].fillna(-1)
data['rating'] = data['rating'].fillna('unknown')

data.isna().sum()

Теперь посмотрим, что делать с годами.

In [ ]:
print('Доля строк с незаполненным годом:', len(data[data['year_of_release'].isna()]) / len(data))

Поскольку доля таких строк всего 1.6%, можем просто их не использовать. То же самое с названиями и жанрами.

In [ ]:
data = data[~data['year_of_release'].isna()]
data = data[~data['name'].isna()]
data = data[~data['genre'].isna()]

data.isna().sum()

Добавим столбец с суммарными продажами во всех регионах.

In [ ]:
data['all_regions_sales'] = data["na_sales"] + data["eu_sales"] + data["jp_sales"] + data["other_sales"]
data.head()

***Вывод:*** Аномалии исправлены, пропуски обработаны, столбец с суммарными продажами во всех регионах добавлен

<a id='explore_data'></a>
# 3. Исследовательский анализ данных

На этом шаге:
* Посмотрим, сколько игр выпускалось в разные годы. Важны ли данные за все периоды?
* Посмотрим, как менялись продажи по платформам. Выберите платформы с наибольшими суммарными продажами и постройте распределение по годам. За какой характерный срок появляются новые и исчезают старые платформы?
* Определим актуальный период для достаточного прогнозирования
* Определим, какие платформы лидируют по продажам, растут или падают? Выбем несколько потенциально прибыльных платформ.
* Построем график «ящик с усами» по глобальным продажам игр в разбивке по платформам.
* Посмотрим, как влияют на продажи внутри одной популярной платформы отзывы пользователей и критиков.
* Построим диаграмму рассеяния и посчитаем корреляцию между отзывами и продажами.
* Соотнесем выводы с продажами игр на других платформах.
* Посмотрим на общее распределение игр по жанрам.


Посмотрим, сколько игр выпускалось в разные годы

In [ ]:
years_counts = data['year_of_release'].value_counts()
years_counts = pd.DataFrame({'year':years_counts.index, 'games_count':years_counts.values})
#display(years_counts.sort_values(by='year'))


years_counts.sort_values(by='year').plot(x='year', y='games_count', kind='bar', title='Количество игр по годам', figsize=(10,5))
max_games_year = years_counts[years_counts['games_count'] == years_counts['games_count'].max()]['year'][0]
print('Максимум игр выпущено в', max_games_year , 'году')
    

***Вывод:*** Общая тенденция на графике видна довольно отчетливо: после 1990х пошло равномерное увеличение кол-ва выпускаемых игр вплоть до 2008, далее пошел аналогичный спад, причем в 2012 очень резкий.

Посмотрим, как менялись продажи по платформам. Выберим платформы с наибольшими суммарными продажами и построим распределение по годам. 

In [ ]:
top_platforms = data.groupby('platform')['all_regions_sales'].sum().sort_values(ascending=False)[:10]
top_platforms = data[data['platform'].isin(top_platforms.index)]

top_platforms.pivot_table(index='year_of_release', columns='platform', values='all_regions_sales', aggfunc='sum').plot(figsize=(15,8))

***Вывод:*** По графикам продаж 10 топовых платформ можно сделать некоторые выводы:
* у большинства платформ график продаж имеют резкий рост продаж и аналогично резкий спад после некоторого периода удержания на высоких позициях продаж. Обычно можно выделить 1 или 2 пиковые точки
* характерная продолжительность жизни большинства платформ окло 8-10 лет
* каждые 5-7 лет появляются новые платформы (но тут слишком мало данных чтобы это точно сказать)

Определим актульный период для прогнозирования

Чтобы узнать актульный период для прогнозирования определим среднее время жизни платформ более точно. Для отсеивания мелких выделим плафтформы, у которых сумма продаж больше 1 млн в год и посчитаем как долго такие платформы держатся на плаву с таким кол-вом продаж.

In [ ]:
# платформы с продажами более 1 млн в год, еще актуальные в 2016
platforms_2016_actual = data.query('year_of_release == 2016 and all_regions_sales >= 1').groupby('platform')['year_of_release'].first()
display(platforms_2016_actual.index)

# платформы с продажами более 1 млн в год и которые уже не актульны в 2016
platfroms_top_sales = data[(~data['platform'].isin(platforms_2016_actual.index)) & (data['all_regions_sales'] >= 1)].groupby('platform')
platforms_billion_sales = (platfroms_top_sales['year_of_release'].max() - platfroms_top_sales['year_of_release'].min()).sort_values(ascending=False)
platforms_billion_sales

Теперь узнаем среднее время жизни таких платформ, исключив PC, как сильно выбивающиеся

In [ ]:
print('Среднее время жизни платформ:', platforms_billion_sales[1:].mean(), 'лет')

***Вывод:*** Среднее время жизни платформ около 6 лет, но поскольку игровая индустрия - очень динамично меняющаяяся, возмем половину от этого - 3 года, значит для нас актуальные данные будут с 2014г.

In [ ]:
platforms_actual = data.query('year_of_release >= 2014')

Опредлим платформы, лидирующие по продажам, растущие и падающие, посмотрим по среднему значению.

In [ ]:
platforms_actual.groupby('platform')['all_regions_sales'].mean().sort_values(ascending=False)

***Вывод:*** Лидируют по продажам с 2014 на данный момент:  PS4, XOne, WiiU, Wii, X360 и 3Ds

Теперь посмортим по графку у кого растут суммарные продажи

In [ ]:
platforms_actual.pivot_table(index='year_of_release', columns='platform', values='all_regions_sales', aggfunc='sum').plot(grid=True, figsize=(15,10))

***Вывод:*** По сравнению с 2014г с 2015г на данный момент продажи упали у всех. самые высокие по продажам на данный момент - PS4 и XOne, еще чуть ниже 3Ds, у большинства остальных платформ все примерно одинаково несладко.

Построим график «ящик с усами» по глобальным продажам игр в разбивке по платформам.

In [ ]:
platforms_actual.boxplot(column='all_regions_sales', by='platform', figsize=(15,7))
plt.ylim(0,1)

plt.show()

***Вывод:*** Посмотрев на боксплоты, можем сказать, что разброс по продажам в каждой платформе довольно сильно различается. Поэтому сравним наиболее типичные продажи по платформам, глядя на медианы. И в этом случае лидирует XOne, чуть ниже идет WiiU и далее почти наравне идут PS4, Wii и X360. PSP и PSV отстают больше всех.

Посмотрим, как влияют на продажи внутри одной популярной платформы, например 'XOne' (как популярную, но не самую) отзывы пользователей и критиков. Также посчитаем корреляцию

In [ ]:
# метод для вывода графика зависимоти и корреляций отзывов и продаж для выбранной платформы
def show_corr_score_sales(name_platform):
    selected_platform = platforms_actual.query('platform == @name_platform & critic_score > -1 & user_score > -1')    
    if len(selected_platform) > 0:
        print(f"Корреляция между отзывами критиков и продажами {name_platform}:", selected_platform['critic_score'].corr(selected_platform['all_regions_sales']))
        print(f"Корреляция между отзывами пользователей и продажами {name_platform}:", selected_platform['user_score'].corr(selected_platform['all_regions_sales']))    
        print('\nГрафики корреляции отзывов и продаж на платформе', name_platform)
        selected_platform.plot(x='critic_score', y='all_regions_sales', kind='scatter') 
        plt.show()
        selected_platform.plot(x='user_score', y='all_regions_sales', kind='scatter')
        plt.show()

In [ ]:
show_corr_score_sales('X360')

Коэффициэнты корреляции говорят о том, что линейая связь практически не прослеживается. А по графикам мы видим, что зависимость продаж от отзывов критиков явно имеется и идет на пользу уже после оценки 60. Оценки пользователей вляиют меньше, и даже с очень низкими оценками есть неплохие продажи.

Проверим наши выводы на других платформах

In [ ]:
for pl in platforms_actual['platform'].unique():
    show_corr_score_sales(pl)

***Вывод:*** Посмотрев все графики и коэффициенты корреляции можено сделать вывод, что наши выводы верны только для платформ: PS3, X360, PS4, XOne, PC. На 3DS, WiiU, PSV влияние оценок пользователей чуть больше. На остальных же платформах слишком мало данных для данной оценки.

Посмотрим на общее распределение игр по жанрам. Для этого рассмотрим медианные продажи:

In [ ]:
platforms_actual['genre'].unique()
platforms_actual.groupby('genre')['all_regions_sales'].agg('median').sort_values(ascending=False).plot(kind='bar', x='genre', y='all_regions_sales', grid=True)

***Вывод:*** видно, что самые прибыльный жанр это 'Shooter' - с большим отрывом. Далее отставнием почти в 3 раза идет Sports и далее с плавным убыванием Platform, Fighting, Role-Playing, Simulation. Самые меньшие продажи выходят от Adventure, Puzzle и Strategy.

**Что получили в данном шаге**
* общая тенденция выпуска игр по годам найдена
* продажи по платформам проанализированы: общая тенденция продаж и ее изменения по годам, найдены платформы с наибольшими суммарными и средними продажами, узнали среднее время жизни платформ
* проанализированы продажи по платформам за выбранный период с 2014г: определены лидеры по продажам за период (по среднему и медианному значениям), узнали у кого растут, а кого падают медианные продажи, 
* на примере одной платформы выяснили влияние отзывов пользователей и критиков на продажи, а также сравнили данные выводы с другими платформами
* узнали распределение по жанрам медианных продаж

<a id='portret'></a>
# 4. Портрет пользователя каждого региона

В этом разделе определим для пользователя каждого региона (NA, EU, JP):
* Самые популярные платформы (топ-5)
* Самые популярные жанры (топ-5)
* Посмотрим влияет ли рейтинг ESRB на продажи в отдельном регионе

Топ 5 популярных платформ

In [ ]:
platforms_actual.head()

In [ ]:
# отображает доли продаж для 3х регионов по переданному полю группировки. Вид - круговые диаграммы
def show_pies_for_regions(grouping_field):
    for region in ['na_sales', 'eu_sales', 'jp_sales']:        
        platforms_actual.groupby(grouping_field)[region].sum().sort_values(ascending=False)[:5].plot.pie(subplots=True, legend=True, figsize=(15,5), autopct='%1.1f%%', title=region)
        plt.show()

In [ ]:
show_pies_for_regions('platform')

***Вывод:*** В разных регионах свои любимчики: в Северной Америке - X360 (33%), в Европе - PS3 (30%), в Японии - 3DS (47%). Интересно, что X360 топовый для Северной Америки, совсем не попал в топ 5 в Японии. При этом PS3 во всех регионах на 1-ом или на 2-ом месте, а PS4 и 3DS вошли в топ 5 всех регионов. XOne входит в 5 лидеров только в Северной Америке.

In [ ]:
show_pies_for_regions('genre')

***Вывод:*** Северная Америка и Европа имеют почти идентичное распределение первых 4х топовых жанров (Action, Shooter, Sports и Role-Playing).
А Япония отличилась: у них Role-Playing на 1м месте (вместо 4го), а также добавились 2 жанра не свойственных двум другим регионам: Fighting и Platform.

Рассмотрим влияние рейтинга ESRB на продажи в отдельном регионе

In [ ]:
platforms_actual.pivot_table(index='rating', values=['na_sales', 'eu_sales', 'jp_sales'], aggfunc='sum').sort_values(by='na_sales',  ascending=False).plot(kind='bar', figsize=(15,8), title='Продажи в зависимости от рейтинга ESRB по регионам')

***Вывод:*** По известным рейтингам можно сказать, что в Северной америке и Европе зависимость продаж от рейтинга примерно одинаковая: рейтинг М явно лидирует, Е определенно имеет 2-ое место, а ретинги E10 и T имеют почти равный приоритет. А вот в Японии лидерство имеет рейтинг T, далее рейтинг E и пости такой же M, E10 - самый непопулярный.

Также очень заметное влияние неизвестных рейтингов, доля даже на глаз очень большая, особенно это сказывается на японских рейтингах. Поэтому очень желательно эти данные узнать, если возможно.

Получается, что в Северной Америке и Европе очень сильно склонны к играм, содержащим насилие и непристойности, а в Японии это прявляется в значительно меньшей степени.

**Что получили в данном шаге**
* узнали, что в разных регионах свои любимые платформы
* узнали распределение по жанрам - Япония особенно отличилась
* узнали влияние рейтинга ESRB на продажи в отдельных регионах. Тут же поняли, что слишком много неизвестных рейтингов, желательно их уточнить

<a id='check_hypothysis'></a>
# 5. Проверка гипотез

В этом разделе проверим утверждения:
* Средние пользовательские рейтинги платформ Xbox One и PC одинаковые
* Средние пользовательские рейтинги жанров Action и Sports разные

Нулевые гипотезы сформулируем так, что в каждом рассматриваемом случае рейтинги равны. А альтернативные - что рейтинги различаются
Проверим нулевые гипотезы и сравним полученный уровень значимости (p-значение) с общепринятым alpha=0.05

***Нулевая гипотеза 1:*** Средние пользовательские рейтинги платформ Xbox One и PC **одинаковые**

***Альтернативная гипотеза 1:*** Средние пользовательские рейтинги платформ Xbox One и PC **различаются**

Проверим. При проверкой гипотез из проверяемых данных также пропуски

In [ ]:
results = st.ttest_ind(platforms_actual.query('platform == "XOne"')['user_score'].dropna() , platforms_actual.query('platform == "PC"')['user_score'].dropna() ,  equal_var = False)
print('p-значение: ', results.pvalue)

***Вывод:*** p-значение намного больше выбранного alpha=0.05, поэтому склоняемся к нулевой гипотезе и утверждению: средние пользовательские рейтинги платформ Xbox One и PC одинаковые

***Нулевая гипотеза 2:*** Средние пользовательские рейтинги жанров Action и Sports **одинаковые**

***Альтернативная гипотеза 2:*** Средние пользовательские рейтинги жанров Action и Sports **разные**

In [ ]:
results = st.ttest_ind(platforms_actual.query('genre == "Action"')['user_score'].dropna(), platforms_actual.query('genre == "Sports"')['user_score'].dropna(), equal_var = False)
print('p-значение: ', results.pvalue)

***Вывод***  p-значение значительно меньше выбранного alpha=0.05, поэтому склоняемся к альтернативной гипотезе и утверждению: средние пользовательские рейтинги жанров Action и Sports разные

<a id='summary'></a>
#  Общий вывод

Выведем отдельно выводы, сделанные выше:
* после 1990х пошло равномерное увеличение кол-ва выпускаемых игр вплоть до 2008-2009, далее пошел аналогичный спад
* у большинства платформ график продаж имеют резкий рост продаж и аналогично резкий спад после некоторого периода удержания на высоких позициях продаж. Обычно можно выделить 1 или 2 пиковые точки
* Среднее время жизни платформ около 6 лет, но поскольку игровая индустрия - очень динамично меняющаяяся, возмем половину от этого - 3 года, значит для нас актуальные данные будут с 2014г.
* Платформы, лидирующе по продажам с 2011 на данный момент:  PS3, X360, PS4, 3DS и XOne
* По сравнению с 2014г с 2015г на данный момент продажи упали у всех. самые высокие по продажам на данный момент - PS4 и XOne, еще чуть ниже 3Ds, у большинства остальных платформ все примерно одинаково несладко.
* Разброс по продажам в каждой платформе довольно сильно различается. Поэтому сравним наиболее типичные продажи по платформам, глядя на медианы. И в этом случае лидирует XOne, чуть ниже идет WiiU и далее почти наравне идут PS4, Wii и X360. PSP и PSV отстают больше всех.
* Зависимость продаж от отзывов критиков явно имеется для платформ: PS3, X360, PS4, XOne, PC. Для них же оценки пользователей влияют меньше, и даже с очень низкими оценками есть неплохие продажи. На 3DS, WiiU, PSV влияние отзывов критиков такое же высокое, а также влияние оценок пользователей  больше, чем у вышеназванных платформ.
* самый прибыльный жанр это 'Shooter' - с большим отрывом. Далее отставнием почти в 3 раза идет Sports и далее с плавным убыванием Platform, Fighting, Role-Playing, Simulation. Самые меньшие продажи выходят от Adventure, Puzzle и Strategy.
* В разных регионах свои любимые платформы: в Северной Америке - X360 (33%), в Европе - PS3 (30%), в Японии - 3DS (47%). Интересно, что X360 топовый для Северной Америки, совсем не попал в топ 5 в Японии. При этом PS3 во всех регионах на 1-ом или на 2-ом месте, а PS4 и 3DS вошли в топ 5 всех регионов. XOne входит в 5 лидеров только в Северной Америке.
* Северная Америка и Европа имеют почти идентичное распределение первых 4х топовых жанров (Action, Shooter, Sports и Role-Playing). А Япония отличилась: у них Role-Playing на 1м месте (вместо 4го), а также добавились 2 жанра не свойственных двум другим регионам: Fighting и Platform.
* в Северной америке и Европе зависимость продаж от рейтинга примерно одинаковая: рейтинг М явно лидирует, Е определенно имеет 2-ое место, а ретинги E10 и T имеют почти равный приоритет. А вот в Японии лидерство делят рейтин E и T, М идет только на 3-ем месте и E10 - самый непопулярный.
* вероятно: средние пользовательские рейтинги платформ Xbox One и PC одинаковые
* вероятно: средние пользовательские рейтинги жанров Action и Sports разные



***Вывод:*** Написанные выше выводы показывают как общие закономерности, так и конкретные показатели, актуальные на данный момент. Например игра вероятно будет иметь успех, если написана для платформы PS4, у нее высоки оценки критиков, она в жанре 'Shooter', продажи осуществляются в Европе, имеет ESRB рейтинг 'M'.

Данные выводы помогут как оценить конкретную игру, так и понять какие параметры важно рассаматривать для поиска новых игры.